# fin article 1

## On le fait pour le daily News 

Dans ce NB, après avoir trouvé les mots les plus significatifs qui donnemt des infos sur l'orientation politique des journaux. Nous allons vérifier si notre modéle réussit à trouver l'orientation politique de the guardian et de daily_news

In [54]:
import pandas as pd 
import os 
from src.utils import * 
from src.data_processing import * 

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option('display.max_columns', None)

In [55]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

Un pretraitement a déjà été effectué. Tous les articles dans ce dataframe portent sur les sujets de la bigtech

In [56]:
df_daily_express = pd.read_csv('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv', encoding='utf-8')

In [57]:
# significant_words = read_input('01. output/df_500_with_slope_coef.csv')
significant_words = pd.read_csv('01. output/df_500_with_slope_coef.csv', encoding='utf-8')

In [58]:
# on quitte la vision 
text = df_daily_express['fulltext'].sum()
len(text)

32341251

In [59]:
text = clean(text, 'bigram')

In [60]:
def count_liste(lili, mot):
    return lili.count(mot)

In [61]:
liste_of_qi2_words = significant_words.columns[5:]

In [62]:
df_journal = pd.DataFrame([[word, count_liste(text, word), significant_words.loc[len(significant_words)-2, word],  significant_words.loc[len(significant_words)-1, word]] for word in liste_of_qi2_words], columns=['mot', 'freq', 'intercept', 'slope_coefficient'])

In [63]:
n = df_journal['freq'].sum()

In [64]:
df_journal['freq_relatif'] = df_journal['freq']/n 

In [65]:
significant_words

,Unnamed: 0,Speaker,party,text,party_bool,hon friend,right hon,hon gentleman,friend member,hon ladi,hon member,european union,unit kingdom,thi hous,make sure,thi countri,northern ireland,labour parti,learn friend,absolut right,thi bill,would like,veri much,friend minist,thank hon,previou govern,local author,hon learn,grate hon,friend ha,british peopl,friend make,thi issu,friend secretari,opposit member,new claus,give way,veri import,import point,thi govern,made clear,thi debat,veri good,shadow minist,pay tribut,good point,veri clear,arm forc,look forward,friend absolut,support bill,take place,member state,friend hi,thi import,member north,deputi speaker,leav european,put place,small busi,hous lord,doe right,want see,make clear,thi matter,ha made,veri happi,mr speaker,point made,lord amend,member christchurch,agre hon,member parliament,around world,thi place,hi constitu,govern commit,tax system,friend agre,work close,mental health,friend rais,thi area,european court,last year,congratul hon,bill read,due cours,neighbourhood plan,car park,point hon,hope hon,move bill,veri care,want ensur,beg move,friend give,thi parliament,leader opposit,second read,want make,leav eu,human right,shadow chancellor,assur hon,law enforc,make progress,import issu,free trade,friend prime,first time,statut book,hon hon,friend know,friend right,continu work,eu law,bill would,amend would,take account,thi year,civil partnership,town centr,great deal,thank right,reason whi,import part,friend say,bill hous,learn ladi,doe hon,reassur hon,incom tax,work togeth,side hous,make point,make good,thi veri,labour member,make import,peopl want,thi point,would say,rais import,stand order,bill provid,ha rais,allow us,hous common,make veri,need ensur,ha done,point rais,high street,come back,ministeri colleagu,say hon,whole hous,across hous,madam deputi,local plan,mr deputi,meet ministeri,local commun,privat member,need make,bill ha,let us,unit state,nation secur,meet later,later today,gentleman know,govern amend,morn meet,addit duti,foreign nation,friend chancellor,colleagu addit,think hon,rais thi,make decis,shall meet,made hon,think right,mani peopl,hous shall,would make,ladi ha,gentleman ha,thi morn,sure hon,uk govern,hi question,devolv administr,one thing,act parliament,england wale,last govern,crimin offenc,commend bill,bill make,help peopl,commun act,mani year,across countri,european commun,privat sector,take action,right rais,north east,member bill,health care,ani way,work hard,gentleman ask,doe agre,scottish govern,think would,give hon,gentleman rais,enabl us,ha taken,crimin justic,bill right,say right,thi subject,grate right,ha alreadi,best way,follow hon,primari legisl,bring forward,great pleasur,wa veri,longterm econom,right way,difficult decis,join hon,huge amount,take thi,work done,pleas say,ladi give,shadow secretari,friend point,know hon,point thi,year ago,one two,right balanc,bear mind,take part,way hon,mobil phone,buri north,withdraw agreement,ladi rais,busi rate,put forward,third read,public financ,absolut clear,corpor tax,support thi,listen care,bill also,member rhondda,make thi,park charg,ensur peopl,right say,commend hous,duti thi,nation interest,right thing,econom plan,public servic,five year,member thi,rule law,gener elect,would want,valu money,beer duti,make easier,recognis import,rural area,part unit,rais issu,east somerset,member shipley,common law,want get,introduc new,veri serious,veri grate,govern support,happi meet,majesti govern,pleasur follow,oil ga,ladi know,would requir,import role,select committe,read third,debat thi,good news,suprem court,hous ha,past year,hard work,look care,member south,creat new,said earlier,provis bill,tabl hon,right across,good thing,bill seek,ensur get,opposit front,gentleman make,nh chariti,bill help,soon possibl,tribut hon,taxpay money,tax rate,work ha,hous agre,royal assent,govern member,import bill,white paper,govern work,ladi make,third time,say thi,may well,becaus want,secretari state,christchurch mr,young

In [66]:
# on enlève les mots qui ont une fréquence nulle car ne sont pas dit dans les articles 
df_journal = df_journal.loc[df_journal['freq'] != 0]

In [67]:
import statsmodels.api as sm

y = df_journal['freq_relatif'] - df_journal['intercept']
x = df_journal['slope_coefficient']

# adding the constant term
x = sm.add_constant(x)

result = sm.OLS(y, x).fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     5.138
Date:                Fri, 12 May 2023   Prob (F-statistic):             0.0240
Time:                        12:32:00   Log-Likelihood:                 1209.7
No. Observations:                 357   AIC:                            -2415.
Df Residuals:                     355   BIC:                            -2408.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0011      0.00

## On le fait pout the guardian 

In [68]:
df_the_guardian = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv', encoding='utf-8')

In [69]:
# on quitte la vision 
text = df_the_guardian['fulltext'].sum()
len(text)

52213541

In [70]:
text = clean(text, 'bigram')

In [71]:
df_journal = pd.DataFrame([[word, count_liste(text, word), significant_words.loc[len(significant_words)-2, word],  significant_words.loc[len(significant_words)-1, word]] for word in liste_of_qi2_words], columns=['mot', 'freq', 'intercept', 'slope_coefficient'])

In [72]:
n = df_journal['freq'].sum()

In [73]:
df_journal['freq_relatif'] = df_journal['freq']/n 

In [74]:
# on enlève les mots qui ont une fréquence nulle car ne sont pas dit dans les articles 
df_journal = df_journal.loc[df_journal['freq'] != 0]

In [75]:
import statsmodels.api as sm

y = df_journal['freq_relatif'] - df_journal['intercept']
x = df_journal['slope_coefficient']

# adding the constant term
x = sm.add_constant(x)

result = sm.OLS(y, x).fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     5.873
Date:                Fri, 12 May 2023   Prob (F-statistic):             0.0158
Time:                        12:42:53   Log-Likelihood:                 1543.1
No. Observations:                 411   AIC:                            -3082.
Df Residuals:                     409   BIC:                            -3074.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0007      0.00